## Sequence Modeling with Hidden Markov Models
In this homework, we will work on sequence data. CpG island detection is a well-known problem in bioinformatics. There are four bases in genomes, A, C, G, and T. In some genomic regions CG bases were observed to be  significantly more common than in other regions. These CG rich regions are called CpG islands. We will not use benchmark data with CpG information. Instead, we will use an HMM model to generate our own data by providing transition and emission probabilties. HMM is a distribution depicted by a graphical model, and it is a generative model. The following diagram shows a 2 state HMM. Emission probabilities are shown in rectangles, and transition probabilities are shown in edge labels. 

<img src="cpg.png" alt="cpg" style="width: 350px;" align="left"/>

### Synthetic Sequence Generation

We define a model and probabilities to generate our own sequence data. The probabilities we chose reflect this biological phenomenon. We will later see that our HMM sequence model will predict the location of CpG islands.


In [1]:
from collections import namedtuple

import numpy as np
import pomegranate

We define our generator model using the transition and emission probabilities in the figure.

In [2]:
def build_model(dist1, dist2, trans, name1, name2):
    """
    Args:
        dist1: pomegranate.DiscreteDistribution
            Non island emission probabilities
        dist2: pomegranate.DiscreteDistribution
            CpG island emission probabilities
        trans: collections.namedtuple
            Transition probabilities
        name1: str
            State 1 name
        name2: str
            State 2 name
    """
    s1 = pomegranate.State(dist1, name=name1)
    s2 = pomegranate.State(dist2, name=name2)

    start_s1, start_s2, s11, s12, s21, s22 = trans

    # create HMM model
    model = pomegranate.HiddenMarkovModel()
    # add states
    model.add_states(s1, s2)

    # add transitions
    model.add_transition(model.start, s1, start_s1)
    model.add_transition(model.start, s2, start_s2)
    model.add_transition(s1, s1, s11)
    model.add_transition(s1, s2, s12)
    model.add_transition(s2, s1, s21)
    model.add_transition(s2, s2, s22)

    model.bake()

    return model


non_island = pomegranate.DiscreteDistribution(
    {"A": 0.25, "C": 0.25, "T": 0.25, "G": 0.25})
island = pomegranate.DiscreteDistribution(
    {"A": 0.10, "C": 0.40, "T": 0.10, "G": 0.40})

trans = {"start_s1": 0.5, "start_s2": 0.5,
         "s11": 0.90, "s12": 0.10, "s21": 0.10, "s22": 0.90}
trans = namedtuple("trans", trans.keys())(**trans)

states = ["Non-Island", "CpG Island"]
model = build_model(non_island, island, trans, *states)

We generate synthetic sequence data using the above model.

In [3]:
def generate_data(model, n, length):
    """
    Samples `n` sequences with `length` from `model`
    """
    return np.array([model.sample(length=length) for i in range(n)])


# We generate 2000 sequences each with length 100
X = generate_data(model, 2000, 100)

We define a new HMM model for CpG island prediction. We will learn the transition and emission probabilities from data.

Note: Training takes about 15 minutes.

In [6]:
cpg_predictor = pomegranate.HiddenMarkovModel()

# if you have multithread computing resource, you can increase n_jobs
cpg_predictor = cpg_predictor.from_samples(distribution=pomegranate.DiscreteDistribution,
                                           n_components=2, X=X, algorithm="baum-welch",
                                           state_names=states,
                                           verbose=True, n_jobs=8)

[1] Improvement: 2177904.6538390648	Time (s): 0.3505
[2] Improvement: 4.881711338763125	Time (s): 0.3555
[3] Improvement: 4.044995095580816	Time (s): 0.3534
[4] Improvement: 3.3857433742377907	Time (s): 0.3908
[5] Improvement: 2.8661423691664822	Time (s): 0.3585
[6] Improvement: 2.457488964719232	Time (s): 0.359
[7] Improvement: 2.13751317106653	Time (s): 0.3574
[8] Improvement: 1.888697431248147	Time (s): 0.3525
[9] Improvement: 1.6971491589793004	Time (s): 0.3491
[10] Improvement: 1.5518005934427492	Time (s): 0.3477
[11] Improvement: 1.443815658451058	Time (s): 0.3508
[12] Improvement: 1.3661364239524119	Time (s): 0.3502
[13] Improvement: 1.3131289656739682	Time (s): 0.3518
[14] Improvement: 1.2803030785289593	Time (s): 0.3517
[15] Improvement: 1.2640887194429524	Time (s): 0.362
[16] Improvement: 1.2616567864315584	Time (s): 0.3782
[17] Improvement: 1.2707751726848073	Time (s): 0.3451
[18] Improvement: 1.2896930295974016	Time (s): 0.3503
[19] Improvement: 1.3170477490639314	Time (s):

In [7]:
ex_seq = X[3]
# transition and emission probabil
trans, ems = cpg_predictor.forward_backward("".join(ex_seq))

You can see the contents of `trans` and `ems` with `print` statements. `trans` matrix is a square matrix and contains the expected number of transitions across each edge in the model. Each row and each column is a state. Direction of transitions is from rows to columns, i.e., `trans_{ij}` contains the expected number of transitions from state in row_i to state in column_j. We have two states related to our problem, CpG Island and Non-Island states and two other states for start and end of the HMM model. Our `ex_seq` has 100 characters in it, sum of the expected transition counts on each edge (`np.sum(trans)`) is equal to 100. 

`ems` is a 100x2 matrix. Each row is an emission, each column is a state. `ems_{ij}` is the natural logarithm of the probability of emitting `ex_seq[i]` in state `j`. Sum of the emission probabilities of each state for `ex_seq[i]` is 1.

In [8]:
print(X[0].shape)
print(ems.shape)
print(trans.shape)
print(trans)
print(ems)
np.sum(trans)
np.sum(np.exp(ems), axis=1)

(100,)
(100, 2)
(4, 4)
[[28.9712244   5.13454797  0.          0.        ]
 [ 4.79652357 60.09770405  0.          0.        ]
 [ 0.54943288  0.45056712  0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[-0.59886867 -0.79724822]
 [-0.50113069 -0.93101168]
 [-0.30015305 -1.34978828]
 [-0.26168137 -1.46861676]
 [-0.32818581 -1.27378452]
 [-0.3070928  -1.33022536]
 [-0.42106233 -1.06812923]
 [-0.88187863 -0.53444303]
 [-1.29445167 -0.32027151]
 [-1.6366328  -0.21645883]
 [-1.88085001 -0.16541778]
 [-2.00370204 -0.14483526]
 [-1.99107829 -0.14681709]
 [-1.84352999 -0.17228148]
 [-1.57871708 -0.23097352]
 [-1.94935518 -0.15357766]
 [-2.24918024 -0.11147435]
 [-2.46868222 -0.08849946]
 [-2.60568823 -0.07672155]
 [-2.66131248 -0.07241638]
 [-2.63776944 -0.07420711]
 [-2.53542779 -0.08254263]
 [-2.34990652 -0.10023819]
 [-2.08237397 -0.13311318]
 [-1.73830778 -0.1933635 ]
 [-2.08132956 -0.13326197]
 [-2.34722606 -0.10052122]
 [-2.5302089  -0.08299297]
 [-2.62867939 -0

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Next, we will print the original and predicted state sequences, as well as the probabilities of each state for each emission.

In [9]:
def predict(model, seq, path, mapping):    
    preds = [state[1].name for state in model.maximum_a_posteriori(seq)[1]]
    preds = [mapping[p] for p in preds]
    path = [mapping[s.name] for s in path]
    
    print("{:10s}: {}".format("Sequence", "".join(seq)))
    print("{:10s}: {}".format("HMM Pred", "".join(map(str, preds))))
    print("{:10s}: {}".format("True Path", "".join(map(str, path))))
    
seq, path = model.sample(n=1, length=100, path=True)[0]

predict(cpg_predictor, seq, path[1:], {"Non-Island": 0, "CpG Island": 1})

Sequence  : GCTGCGCTTTGGACCCGTGCACCGGGCTGCGGTGGTGGGGCGCCGGCCCGTACGGGTGCCGGCGCTGAAGTCGGCCGCTGCAGGGGTGTGACTGGTACAG
HMM Pred  : 0000000111111000000000000000000000000000000000000000000000000000011111100000000000000011111111111111
True Path : 1001100000001111100111111000111111111111111111111100001111111111111100000011111111110011100111111111


/var/folders/yb/xfw1ct9d38v_f9wpjpqrzc4m0000gq/T/ipykernel_67047/361697912.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq, path = model.sample(n=1, length=100, path=True)[0]


### Exercise

This exercise was copied from Dr. Ziv Bar-Joseph lecture notes.

Given model below and the sequence *x*="123412316261636461623411221341", define an HMM model with the given model probabilities to answer the following questions:

1. What is the likelihood of observing sequence *x* given model probabilities? (**Hint: Use forward algorithm**)
2. Is it more likely that the 3rd observed “6” in *x* comes from a Loaded or a Fair die? 
 (**Hint: Use Viterbi algorithm**)
 
You can check the [API documentation](https://pomegranate.readthedocs.io/en/latest/HiddenMarkovModel.html#module-pomegranate.hmm).
 
<img src="casino.png" alt="casino" style="width: 300px;" align="left"/>

In [10]:
non_island = pomegranate.DiscreteDistribution(
    {"1": 1 / 6, "2": 1 / 6, "3": 1 / 6, "4": 1 / 6, "5": 1 / 6, "6": 1 / 6})
island = pomegranate.DiscreteDistribution(
    {"1": 1 / 10, "2": 1 / 10, "3": 1 / 10, "4": 1 / 10, "5": 1 / 10, "6": 1 / 2})

trans = {"start_s1": 0.5, "start_s2": 0.5,
         "s11": 0.95, "s12": 0.05, "s21": 0.10, "s22": 0.90}
trans = namedtuple("trans", trans.keys())(**trans)

states = ["Fair", "Loaded"]
model = build_model(non_island, island, trans, *states)

x = "123412316261636461623411221341"

In [11]:
# Generate 2000 sequences
X = generate_data(model, 2000, len(x))

In [12]:
dice_predictor = pomegranate.HiddenMarkovModel()

# if you have multithread computing resource, you can increase n_jobs
dice_predictor = dice_predictor.from_samples(distribution=pomegranate.DiscreteDistribution,
                                           n_components=2, X=X, algorithm="baum-welch",
                                           state_names=states,
                                           verbose=True, n_jobs=8)

[1] Improvement: 575419.6200563316	Time (s): 0.2891
[2] Improvement: 2.89049396077462	Time (s): 0.1952
[3] Improvement: 2.8997331104037585	Time (s): 0.2073
[4] Improvement: 2.9448357180372113	Time (s): 0.1919
[5] Improvement: 3.025162397301756	Time (s): 0.1962
[6] Improvement: 3.141008357197279	Time (s): 0.1949
[7] Improvement: 3.2935221639781957	Time (s): 0.2046
[8] Improvement: 3.484660714180791	Time (s): 0.1946
[9] Improvement: 3.717170772390091	Time (s): 0.2106
[10] Improvement: 3.994588889865554	Time (s): 0.1934
[11] Improvement: 4.321251800938626	Time (s): 0.1964
[12] Improvement: 4.702308428357355	Time (s): 0.1973
[13] Improvement: 5.143722549750237	Time (s): 0.2018
[14] Improvement: 5.652251561419689	Time (s): 0.1769
[15] Improvement: 6.2353816999384435	Time (s): 0.206
[16] Improvement: 6.901193105935818	Time (s): 0.2038
[17] Improvement: 7.658119420695584	Time (s): 0.2748
[18] Improvement: 8.514556382986484	Time (s): 0.2246
[19] Improvement: 9.478263342534774	Time (s): 0.1949


In [45]:
# Since we just want the forward probability we need to remove log
np.exp(dice_predictor.log_probability(x))

3.648663382277969e-24

In [26]:
# model.predict(x, algorithm='viterbi')
# dice_predictor.predict(x, algorithm='viterbi')
dice_predictor.viterbi(x)

(-56.20291144433178,
 [(2, {
       "class" : "State",
       "distribution" : null,
       "name" : "None-start",
       "weight" : 1.0
   }),
  (1,
   {
       "class" : "State",
       "distribution" : {
           "class" : "Distribution",
           "dtype" : "numpy.str_",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "1" : 0.16903197859849442,
                   "2" : 0.168228661368217,
                   "3" : 0.16479317917729605,
                   "4" : 0.16837118003205334,
                   "5" : 0.1689394526915753,
                   "6" : 0.1606355481323619
               }
           ],
           "frozen" : false
       },
       "name" : "Loaded",
       "weight" : 1.0
   }),
  (1,
   {
       "class" : "State",
       "distribution" : {
           "class" : "Distribution",
           "dtype" : "numpy.str_",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
             

During our tests for part 2 we found that the 3rd 6 to be rolled will always be more likely to come from a fair die according to the viterbi algorithm. This is because there are higher probability transitions going into the fair die state which causes the viterbi path to be made up of either all fair die or one loaded die to start and all fair die.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

Please contact Haluk Dogan (<a href="mailto:hdogan@vivaldi.net">hdogan@vivaldi.net</a>) for further questions or inquries.